In [1]:
!pip install fastapi uvicorn pyngrok pydub faster-whisper openai
!apt-get install ffmpeg
!pip install python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
%%writefile inference.py
import logging
import re
from pathlib import Path

from fastapi import FastAPI, HTTPException, UploadFile, File
from fastapi.responses import JSONResponse
import torch
from faster_whisper import WhisperModel
from openai import OpenAI

# ============ Logging ============
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ============ API Key & Directories ============
OPENAI_API_KEY = ""
client = OpenAI(api_key=OPENAI_API_KEY)

TMP_WAV = Path("/tmp/temp.wav")

# ============ Model Initialization ============
device = "cuda" if torch.cuda.is_available() else "cpu"
model = WhisperModel("large-v3", device=device, compute_type="float32")

EGYPTIAN_PROMPT = (
    "This is Egyptian Arabic speech. Use dialect words like 'aywa', 'la2', 'mesh', "
    "'enta fein', '3ayez', 'hat', 'mashy'. Transcribe naturally as Egyptians speak."
)

# ============ GPT Helpers ============

def gpt_translate_text(arabic_text: str) -> str:
    system_prompt = (
        "You are a professional translator specializing in Egyptian Arabic to English."
        " Translate the following transcript accurately into clear, natural English, preserving speaker intent."
    )
    try:
        logger.info("🧠 Translating via GPT")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": arabic_text}
            ],
            temperature=0.0,
            max_tokens=2000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logger.error("❌ GPT Translation Error: %s", str(e))
        return f"(Translation Error): {str(e)}"


def gpt_label_speakers(dialogue: str) -> str:
    system_prompt = (
        "You're a smart assistant for real estate calls."
        "The following is a transcript of a conversation between a real estate Broker and a potential Client."
        "Label each line clearly as 'Broker:' or 'Client:' based on who would logically speak the line in a property inquiry."
        "Assume the Client is the one asking questions about services or features, and the Broker is providing answers or confirmations."
        "Do not change the content. Just add the correct speaker labels."
    )
    try:
        logger.info("🧠 Labeling speakers via GPT")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": dialogue}
            ],
            temperature=0.3,
            max_tokens=2000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logger.error("❌ GPT Speaker Labeling Error: %s", str(e))
        return f"(Labeling Error): {str(e)}"


def gpt_rephrase_full(text: str) -> str:
    system_prompt = (
        "You are a professional real estate sales assistant.\n"
        "Rephrase each line individually to be more polite, fluent, and domain-specific for real estate sales.\n"
        "Keep 'Client:' and 'Broker:' labels exactly as they are.\n"
        "Rewrite in clear business English.\n"
    )
    try:
        logger.info("🧠 Rephrasing via GPT")
        resp = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": text}
            ],
            temperature=0.4,
            max_tokens=2000
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        logger.error("❌ GPT Rephrasing Error: %s", str(e))
        return f"(Rephrasing Error): {str(e)}"

# ============ Core Logic ============
def transcribe_logic(wav_path: Path) -> dict:
    # Step 1: Transcribe Arabic
    params = dict(
        beam_size=2, vad_filter=True,
        vad_parameters={"min_silence_duration_ms": 500},
        temperature=0.0, condition_on_previous_text=False,
        compression_ratio_threshold=2.4, language="ar",
        best_of=1, word_timestamps=False,
        initial_prompt=EGYPTIAN_PROMPT
    )
    segments, info = model.transcribe(str(wav_path), **params)
    raw_arabic = "\n".join(seg.text.strip() for seg in segments)

    # Step 2: Translate via GPT for better accuracy
    translated_clean = gpt_translate_text(raw_arabic)

    # Step 3: Let GPT assign speaker roles
    labeled = gpt_label_speakers(translated_clean)

    # Step 4: Let GPT rephrase as polished dialogue
    rephrased = gpt_rephrase_full(labeled)

    return {
        "status": "success",
        "language": info.language,
        "transcription": raw_arabic,
        "translation": translated_clean,
        "labeled_dialogue": labeled,
        "final_rephrased_text": rephrased
    }

# ============ FastAPI App ============
app = FastAPI()

@app.get("/ping")
def ping():
    return JSONResponse({"status": "ok"})

@app.post("/invocations")
async def invoke(
    file: UploadFile = File(...)
):
    try:
        content = await file.read()
        TMP_WAV.write_bytes(content)
        result = transcribe_logic(TMP_WAV)
        return JSONResponse(result)
    except Exception as e:
        logger.error("Invocation error: %s", e)
        raise HTTPException(status_code=500, detail="Internal error")


Writing inference.py


In [3]:
!ngrok config add-authtoken 2wECNofpyItvbpfFXHvTDvrrAHh_6V4BeKEqH5aNZvQcn6zys

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
# in a new Colab cell
!pkill -f uvicorn


In [ ]:
# Colab cell continued: ngrok + Uvicorn startup

from pyngrok import ngrok
import nest_asyncio, threading, uvicorn
from inference import app  # your FastAPI app

# 1) Kill any existing tunnels
ngrok.kill()

# 2) (Re-)authenticate
ngrok.set_auth_token("")

# 3) Allow nested loops in Colab
nest_asyncio.apply()

# 4) Open an HTTP-only tunnel on port 8000
tunnel = ngrok.connect(
    8000,
    proto="http",            # make it an HTTP tunnel
    bind_tls=False           # disable the TLS (HTTPS) endpoint
)
print("→ HTTP tunnel URL:", tunnel.public_url)

# 5) Start Uvicorn as before
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run_server, daemon=True).start()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

→ HTTP tunnel URL: http://cfc5-34-16-136-168.ngrok-free.app


In [ ]:
import requests

resp = requests.get(f"{public_url}/ping")
print(resp.status_code, resp.json())


INFO:     34.125.179.44:0 - "GET /ping HTTP/1.1" 200 OK
200 {'status': 'ok'}


In [ ]:
with open("/content/call_2_trimmed.wav", "rb") as f:
    resp = requests.post(
        f"{public_url}/invocations",
        files={"file": ("call.wav", f, "audio/wav")},
        data={"emp_id": "Broker"}
    )
    print(resp.status_code, resp.json())


INFO:     34.125.179.44:0 - "POST /invocations HTTP/1.1" 200 OK
200 {'status': 'success', 'language': 'ar', 'transcription': '[3.82-4.82] Client:  السلام عليكم.\n[4.82-6.11] Client:  السلام عليكم.\n[6.11-9.90] Client:  مين معايا؟\n[9.90-12.90] Client:  مع حضرتك كريم من شركة Property Finder.\n[12.90-15.06] Client:  زكا كريم عامل ايه؟\n[15.06-23.06] Client:  الحمد لله. حضرتك بس كنت مليت فورم في الحملة التسويقية بتاعتنا بخصوص وحدة في العاصمة الإدارية الجديدة.\n[23.06-27.06] Client:  فحبيت أتابع مع حضرتك لو مفيش مانع.\n[27.06-30.19] Client:  تمام. قول. قول اللي عندك.\n[30.19-33.48] Client:  أولا شكرا جدا على اهتمام حضرتك\n[33.48-36.08] Client:  أنا بس حابب أبدأ بخطوة بسيطة\n[36.08-38.76] Client:  عشان أشرح لك المشاريع\n[38.76-41.54] Client:  ونحاول نوصل لأنسب مشروع في يوم\n[41.54-43.24] Client:  بالنسبة للمنطقة\n[43.24-49.32] Client:  حضرتك محدد حي معين أو كومباونت معين في العاصمة؟\n[49.32-52.24] Client:  والله أنا كنت بدور في الـ R8 أو الـ R7\n[52.24-54.52] Client:  بس ما عنديش منع أشوف ا